# What is the word embedding?

An Embedding is a mapping from discrete object into vectors of real numbers. It is important to use text on task of Natural lanugauges processing, in particular, Machine learning and Deep Learning. For an example, When you embed words into vector space, It looks as follows:

```
blue:  (0.01359, 0.00075997, 0.24608, ..., -0.2524, 1.0048, 0.06259)
blues:  (0.01396, 0.11887, -0.48963, ..., 0.033483, -0.10007, 0.1158)
orange:  (-0.24776, -0.12359, 0.20986, ..., 0.079717, 0.23865, -0.014213)
oranges:  (-0.35609, 0.21854, 0.080944, ..., -0.35413, 0.38511, -0.070976)
```

In case of vetor of word embedding, Specifically, We don't know what each dimension in a vector means. However, We can realize some features from overall pattern of location and distance between vectors that Machine Learning take advantage of. 

Because words of text can be represented as natural vectors,the way of word embedding is standard and effective way to transform such discrete. depending on how dense it is, and how better it utilize on the task that need vector of words of text.

# How to embed word in Tensorflow

To create word embeddings in Tensorflow, we first split the text into words and then we assign integer number to every word in every vocabulary as follow

```python
text = "I have a cat."
# array has Index of each words.The Index is word_ids 
# Index:  0,  1  , 2,  3 , 4  --> word_ids
# Array: [I, have, a, cat, .]
array_text = [I, have, a, cat, .]
```

As you can see above, We need to map **word_ids** to **vectors** in Tensorflow. For this situation, use the following functions :

```python
word_embeddings = tf.get_variable("word_embeddings", [vocabulary_size, embedding_size])
embedded_word_ids = tf.nn.embedding_lookup(word_embeddings, word_ids)
```

The variable of **word_embeddings** is a array that is comprised of **total vocabulary size** and the dimesional size of each words we called **embedding size**. For example. 

 - a sentence(a document) : I have a cat. 
 - The total size of vocabularies : 5(I, have, a, cat, .)
 - The size of embedding : whatever size you want, 300-dimensional for each words that normally used.

In detail, Let's walk through the function of **tf.nn.embedding_lookup**.

```python
word_embeddings = tf.get_variable("word_embeddings", [vocabulary_size, embedding_size])
embedded_word_ids = tf.nn.embedding_lookup(word_embeddings, word_ids)
```

This function, **tf.nn.embedding_lookup**, literally means lookup in embeddings array depending on **word_ids**.

let's say, we have word_emedddings matrix like this:

```python
# word embedding function of tensorflow
# word_embeddings is 2 by 2 matrix, each factor is randomized. 
# [[00, 01],
#  [10, 11]]
word_embeddings = tf.get_variable("word_embeddings_test", [2, 2])
# In here, 1 mean the first row of word_embedding matrix. 
embedded_word_ids = tf.nn.embedding_lookup(word_embeddings, ids=1)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    embedding_result_test, embedded_word_result_test = sess.run([word_embeddings, embedded_word_ids])
    print("=== the test result ===\n")
    print("The result of word_embedding:\n",embedding_result_test, "\n")
    print("The result of embedded_word_ids:\n",embedded_word_result_test, "\n")
```

If you execute the graph above, the result of **tf.nn.embedding_lookup(matrix, ids)** is rows equal to **ids**. Let's say ids set **1** as ids. the 1 index row of **word_embeddings** matrix  is returned. If you enter ids such as a list([0, 1]), the return value is a list including row value of **word_embeddings** matrix depending the factors of list. 

  - If ids is 1, the return value is word_embedding[1]
  
  - If ids is [0, 1], the return value is [word_embedding[0], word_embedding[1]]
  
Let's see the result of computation graph above:

```shell
=== the test result ===

The result of word_embedding:
 [[-0.7246539   0.43106437]
 [ 0.52070427  1.06491148]] 

The result of embedded_word_ids:
 [ 0.52070427  1.06491148] 
```

# How to use Tensorflow projector as debugging. 

When you are embedding text or image with Tensorflow, Tensorflow provide great tool to help you easily debug. It is calle Tensorboard.

[Tensorboard](https://www.youtube.com/embed/eBbEDRsCmv4) is great tool. that draws your graph of computation and help you check some value of your model like FeedForward Neural Network. But, here you walk through how to project embedding matrix into 2-D ro 3-D.

let's see an simple example code:

First is no label 

```python 
# How to use tensorboard for WordEmbedding. 
import os
import tensorflow as tf

LOG_DIR = os.path.join(os.getcwd(),"log/")

print("the current working directory:", LOG_DIR, "\n")

# word vector of embedding. 
embedding_temp = tf.Variable([[0.1,0.2],[0.5,0.5]], dtype=tf.float32, name="test_embedding-no-label")

# To initilize the variable. 
init = tf.global_variables_initializer()

# To add ops to save and restore all the variable
# When you want to make it
saver = tf.train.Saver()

with tf.Session() as sess:
    
    sess.run(init)
    
    # print embedding_temp
    embedding_temp_ = sess.run(embedding_temp)
    print(embedding_temp_)
    
    save_path = saver.save(sess, os.path.join(LOG_DIR, "test_embedding_model.ckpt"), global_step=0)
    print("\nModel Saved in file: %s" % save_path)
```

In here, you have to save checkpoint file to save all the variables on your model. if you completely save checkpoint file, move to directory you saved checkpoint file to. 

and then prompt in command line like this:

> tensorboard --logdir=the-path-you-saved-checkpoint-file-to

the result of the above code is: 

![](https://raw.githubusercontent.com/hyunyoung2/hyunyoung2_Machine_Learning/master/Tutorial/Tensorflow/03.Word2Vec/images/01.Word_Embedding/Word_Embedding_With_no_label_on_tensorboard.png)

As you can see the code, there is no label for each words. so on Tensorboard Projector, each words are represented as index number.

Let's say another one using with lable of embedding. In particular, here I tested it for NLP. 

```python
# How to use tensorboard for WordEmbedding. 
# with metadata file 
import os
import tensorflow as tf

LOG_DIR = os.path.join(os.getcwd(),"log/")

print("the current working directory:", LOG_DIR, "\n")

LABEL_FILE = os.path.join(os.getcwd(), "log/lables.tsv")

print("the current LABEL_FILE:", LABEL_FILE, "\n")

LABEL_NUM = 2
LABEL = ["NLP", "Deep Learning"]

# Write label file 
with open(LABEL_FILE,"w") as f:
    for i in range(LABEL_NUM):
        f.write(LABEL[i]+"\n")
    print("labels.tsv file Created!\n")

# word vector variable to embedding. 
embedding_temp = tf.Variable([[0.1,0.2],[0.5,0.5]], dtype=tf.float32, name="test_embedding")

# To initilize the variable. 
init = tf.global_variables_initializer()

# To add ops to save and restore all the variable
# When you want to make it
saver = tf.train.Saver()

with tf.Session() as sess:
    
    sess.run(init)
    
    # print embedding_temp
    embedding_temp_ = sess.run(embedding_temp)
    print(embedding_temp_)
    
    # Format: tensorflow/contrib/tensorboard/plugins/projector/projector_config.proto
    config = tf.contrib.tensorboard.plugins.projector.ProjectorConfig()
    # You can add multiple embeddings. Here we add only one.
    embedding_config = config.embeddings.add()
    embedding_config.tensor_name = embedding_temp.name
    # Link this tensor to its metadata file (e.g. labels).
    embedding_config.metadata_path = LABEL_FILE
    
    # Use the same LOG_DIR where you stored your checkpoint.
    summary_writer = tf.summary.FileWriter(LOG_DIR)
    
    # The next line writes a projector_config.pbtxt in the LOG_DtenIR. TensorBoard will
    # read this file during startup.
    tf.contrib.tensorboard.plugins.projector.visualize_embeddings(summary_writer, config)
    
    
    save_path = saver.save(sess, os.path.join(LOG_DIR, "test_embedding_model.ckpt"), global_step=1)
    print("\nModel Saved in file: %s" % save_path)
```

The different one from the former one is **tf.contrib.tensorboard.plugins.projector.ProjectorConfig()**.

you have to set labal file(TSV file) up to Its return value(**config**) 

If you create label file, the format of the file isto write words line by line depending on word_embedding matrix like this:

```
NLP\n
Deep Lerning\n
```

![](https://raw.githubusercontent.com/hyunyoung2/hyunyoung2_Machine_Learning/master/Tutorial/Tensorflow/03.Word2Vec/images/01.Word_Embedding/Word_Embedding_With_label_on_tensorboard.png)


In order to use Tensorboard projector, First you need variable to represent embedding data like **embedding_temp** on the above codes. And then just save checkpoint file to save all the variable of your model. 

It  is all what you have to do for projector of embeddin onto Tensorboard. 

If you save checkpoint file, run the following:

> tensorboard --logdir=the-path-you-saved-checkpoint-file-to

# Reference 

   - [Tensorflow's Embeddings](https://www.tensorflow.org/programmers_guide/embedding)   

   - [Tensorflow's how to use Tensorboard for Embedding r1.0](https://www.tensorflow.org/versions/r1.0/get_started/embedding_viz)
      
   - [Tensorflow's API of tf.nn.embedding_lookup](https://www.tensorflow.org/api_docs/python/tf/nn/embedding_lookup)  

   - [Tensorflow's API of tf.gather](https://www.tensorflow.org/api_docs/python/tf/gather)  

   - [Tensorflow's tensor](https://www.tensorflow.org/programmers_guide/tensors)  

   - [Tensorflow's variable](https://www.tensorflow.org/programmers_guide/variables)  

   - [an example code of Tensorboard](https://github.com/dandelionmane/tf-dev-summit-tensorboard-tutorial)

In [1]:
import sys
import tensorflow as tf

# -- The version of python is 3.5.2---
print("The version of python:", sys.version, "\n")
# -- The version of tensorflow is 1.4 --
print("The version of tenserflow:", tf.__version__)

The version of python: 3.5.2 (default, Nov 23 2017, 16:37:01) 
[GCC 5.4.0 20160609] 

The version of tenserflow: 1.4.0


In [2]:
# Let's see how to use word embedding function in tensorflow
import collections # for Counter class

sample_string = ["I like deep learning", 
                "I like NLP",
                "I enjoy flying"]

vocabulary_size = 0 
embbeding_size = 2
# To build up dataset without duplication of words like this :
# word2id, id2word, word_count, total vocab 
def build_up_dataset(sentences):
    # merger a variety of strings into one string, delimiter=" "(white space) 
    words = " ".join(sample_string).split()
    # without duplication of words, each words couting. 
    counter = collections.Counter(words).most_common()
    temp_list_id2word = [items[0] for items in counter] # idx-> word : search word from index
    # search idx from word
    dict_temp_word2id = {key:val for key, val in enumerate(temp_list_id2word)}
    # total of vocabulary
    voca_size = len(temp_list_id2word)
    return temp_list_id2word, dict_temp_word2id, counter, voca_size
    
list_id2word, dict_word2id, count, vocabulary_size = build_up_dataset(sample_string)

print("id2word:", list_id2word, "\n")
print("word2id:", dict_word2id, "\n")
print("count of word:\n", count, "\n")
print("the size of vocabularay:", vocabulary_size, "\n")

id2word: ['I', 'like', 'deep', 'flying', 'NLP', 'learning', 'enjoy'] 

word2id: {0: 'I', 1: 'like', 2: 'deep', 3: 'flying', 4: 'NLP', 5: 'learning', 6: 'enjoy'} 

count of word:
 [('I', 3), ('like', 2), ('deep', 1), ('flying', 1), ('NLP', 1), ('learning', 1), ('enjoy', 1)] 

the size of vocabularay: 7 



In [3]:
# word embedding function of tensorflow
word_ids= tf.placeholder(tf.int32, shape=[1,2,2])
word_embeddings = tf.get_variable("word_embeddings", [vocabulary_size, embbeding_size])
embedded_word_ids = tf.nn.embedding_lookup(word_embeddings, word_ids)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    word_ids, embedding_result, embedded_word_result = sess.run([word_ids, word_embeddings, embedded_word_ids], feed_dict={word_ids:[[[0,1],[2,2]]]})
    print("=== the test result ===\n")
    print("The result of word_ids:\n",word_ids, "\n")
    print("The result of word_embedding:\n",embedding_result, "\n")
    print("The result of embedded_word_ids:\n",embedded_word_result, "\n")

=== the test result ===

The result of word_ids:
 [[[0 1]
  [2 2]]] 

The result of word_embedding:
 [[-0.10185999 -0.59124708]
 [ 0.30318129 -0.42134538]
 [-0.68222558  0.21481109]
 [-0.14251018 -0.52396905]
 [-0.64809692 -0.79607165]
 [-0.36452004 -0.47127265]
 [ 0.04024273 -0.38521987]] 

The result of embedded_word_ids:
 [[[[-0.10185999 -0.59124708]
   [ 0.30318129 -0.42134538]]

  [[-0.68222558  0.21481109]
   [-0.68222558  0.21481109]]]] 



In [4]:
# word embedding function of tensorflow
word_embeddings = tf.get_variable("word_embeddings_test", [2, 2])
embedded_word_ids = tf.nn.embedding_lookup(word_embeddings, 1)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    embedding_result_test, embedded_word_result_test = sess.run([word_embeddings, embedded_word_ids])
    print("=== the test result ===\n")
    print("The result of word_embedding:\n",embedding_result_test, "\n")
    print("The result of embedded_word_ids:\n",embedded_word_result_test, "\n")

=== the test result ===

The result of word_embedding:
 [[ 0.0130285  -1.20233023]
 [-0.60363257  0.98856938]] 

The result of embedded_word_ids:
 [-0.60363257  0.98856938] 



In [5]:
# How to use tensorboard for WordEmbedding. 
import os
import tensorflow as tf

LOG_DIR = os.path.join(os.getcwd(),"log/")

print("the current working directory:", LOG_DIR, "\n")

# word vector of embedding. 
embedding_temp = tf.Variable([[0.1,0.2],[0.5,0.5]], dtype=tf.float32, name="test_embedding-no-label")

# To initilize the variable. 
init = tf.global_variables_initializer()

# To add ops to save and restore all the variable
# When you want to make it
saver = tf.train.Saver()

with tf.Session() as sess:
    
    sess.run(init)
    
    # print embedding_temp
    embedding_temp_ = sess.run(embedding_temp)
    print(embedding_temp_)
    
    save_path = saver.save(sess, os.path.join(LOG_DIR, "test_embedding_model.ckpt"), global_step=0)
    print("\nModel Saved in file: %s" % save_path)

the current working directory: /home/hyunyoung2/My_lab/hyunyoung2_Machine_Learning/Tutorial/Tensorflow/03.Word2Vec/log/ 

[[ 0.1  0.2]
 [ 0.5  0.5]]

Model Saved in file: /home/hyunyoung2/My_lab/hyunyoung2_Machine_Learning/Tutorial/Tensorflow/03.Word2Vec/log/test_embedding_model.ckpt-0


In [6]:
# How to use tensorboard for WordEmbedding. 
# with metadata file 
import os
import tensorflow as tf

LOG_DIR = os.path.join(os.getcwd(),"log/")

print("the current working directory:", LOG_DIR, "\n")

LABEL_FILE = os.path.join(os.getcwd(), "log/lables.tsv")

print("the current LABEL_FILE:", LABEL_FILE, "\n")

LABEL_NUM = 2
LABEL = ["NLP", "Deep Learning"]

# Write label file 
with open(LABEL_FILE,"w") as f:
    for i in range(LABEL_NUM):
        f.write(LABEL[i]+"\n")
    print("labels.tsv file Created!\n")

# word vector of embedding. 
embedding_temp = tf.Variable([[0.1,0.2],[0.5,0.5]], dtype=tf.float32, name="test_embedding")

# To initilize the variable. 
init = tf.global_variables_initializer()

# To add ops to save and restore all the variable
# When you want to make it
saver = tf.train.Saver()

with tf.Session() as sess:
    
    sess.run(init)
    
    # print embedding_temp
    embedding_temp_ = sess.run(embedding_temp)
    print(embedding_temp_)
    
    # Format: tensorflow/contrib/tensorboard/plugins/projector/projector_config.proto
    config = tf.contrib.tensorboard.plugins.projector.ProjectorConfig()
    # You can add multiple embeddings. Here we add only one.
    embedding_config = config.embeddings.add()
    embedding_config.tensor_name = embedding_temp.name
    # Link this tensor to its metadata file (e.g. labels).
    embedding_config.metadata_path = LABEL_FILE
    
    # Use the same LOG_DIR where you stored your checkpoint.
    summary_writer = tf.summary.FileWriter(LOG_DIR)
    
    # The next line writes a projector_config.pbtxt in the LOG_DtenIR. TensorBoard will
    # read this file during startup.
    tf.contrib.tensorboard.plugins.projector.visualize_embeddings(summary_writer, config)
    
    
    save_path = saver.save(sess, os.path.join(LOG_DIR, "test_embedding_model.ckpt"), global_step=1)
    print("\nModel Saved in file: %s" % save_path)

the current working directory: /home/hyunyoung2/My_lab/hyunyoung2_Machine_Learning/Tutorial/Tensorflow/03.Word2Vec/log/ 

the current LABEL_FILE: /home/hyunyoung2/My_lab/hyunyoung2_Machine_Learning/Tutorial/Tensorflow/03.Word2Vec/log/lables.tsv 

labels.tsv file Created!

[[ 0.1  0.2]
 [ 0.5  0.5]]

Model Saved in file: /home/hyunyoung2/My_lab/hyunyoung2_Machine_Learning/Tutorial/Tensorflow/03.Word2Vec/log/test_embedding_model.ckpt-1
